In [1]:
# Environment Import
from experiment_graph.execution_environment import ExecutionEnvironment
from experiment_graph.graph.operations import UserDefinedFunction


/Users/bede01/Documents/work/phd-papers/published/ml-workload-optimization/code/collaborative-optimizer/experiment_graph/graph/node.py:49: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if k is 'execution_environment':


In [2]:
class ClipDataset(UserDefinedFunction):
    def __init__(self, lower=None, upper=None, axis=None):
        super().__init__(return_type='Dataset')
        self.lower = lower
        self.upper= upper
        self.axis = axis
        
    def run(self, underlying_data):
        # here the underlying_data is a pandas dataframe and we are directly calling the pandas clip function
        return underlying_data.clip(lower=self.lower, upper=self.upper, axis=self.axis)

In [12]:
class CustomProject(UserDefinedFunction):
    def __init__(self, column):
        super().__init__(return_type='Feature')
        self.column = column
        
    def run(self, underlying_data):
        # here the underlying_data is a pandas dataframe but the udf is expected to return a Feature (i.e., dataseries)
        return underlying_data[self.column]

In [13]:
class MedianFeature(UserDefinedFunction):
    def __init__(self):
        super().__init__(return_type='Agg')
        
    def run(self, underlying_data):
        # here the underlying_data is pandas data series
        return underlying_data.median()

In [14]:
execution_environment = ExecutionEnvironment()
data = execution_environment.load('./data/sample.csv')

creating a new root node


In [15]:
clip_oper = ClipDataset(lower=2,upper=4)
clipped = data.run_udf(clip_oper)

In [16]:
clipped.data()

,a,b,c,d
0,2,2,3,4
1,4,4,4,4
2,2,2,2,3


In [17]:
projection = CustomProject('a')
feature_a = clipped.run_udf(projection)

In [18]:
feature_a.data()

0    2
1    4
2    2
Name: a, dtype: int64

In [19]:
median_feature=MedianFeature()
median_value = feature_a.run_udf(median_feature)

In [20]:
median_value.data()

2.0